In [1]:
from sklearn.svm import SVC, SVR
from sklearn.model_selection import (StratifiedKFold, RepeatedStratifiedKFold, 
                                     KFold, RepeatedKFold)
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (roc_auc_score, balanced_accuracy_score, recall_score, 
                             brier_score_loss, mean_absolute_error, mean_squared_error, 
                             median_absolute_error, r2_score)
from sklearn.utils.extmath import row_norms
import numpy as np 
import pandas as pd
import nibabel as nib
import os.path as osp
import matplotlib.pyplot as plt
from time import time
from tqdm.notebook import tqdm

In [2]:
def load_nifti(file_path, dtype=np.float):
    return np.array(nib.load(file_path).dataobj, dtype=dtype)


def load_scalar_momenta(scl_mom_folder, patient_ids, mask):
    X_gm_scl = np.zeros((patient_ids.size, mask.sum()))
    X_wm_scl = np.zeros((patient_ids.size, mask.sum()))
    
    print('Loading Scalar Momenta...')
    for i_pat, patient_id, in enumerate(tqdm(patient_ids)):
        tmp = load_nifti(osp.join(scl_mom_folder, 'a_rc1{}T1.nii.gz'.format(patient_id)))
        X_gm_scl[i_pat] = tmp[..., 0][mask]
        X_wm_scl[i_pat] = tmp[..., 1][mask]
    return X_gm_scl, X_wm_scl

In [3]:
regress_covariates = True

if regress_covariates:
    suffix_cov_reg = 'with_covariate_regr'
else:
    suffix_cov_reg = 'without_covariate_regr'

project_folder = '/data/shared/OCDDBSpred'
palm_analysis =  osp.join(project_folder, 'data', 'analysis_data', 
                          'cat12_analysis', 'palm_analysis_s8')
scalar_momenta_folder = osp.join(project_folder, 'data', 'preprocessed')
regr_folder = osp.join(palm_analysis, 'ybocs_1y')
clf_folder = osp.join(palm_analysis, 'responders_vs_nonresponders')

# The data/masks in whole_brain/ROI folders are the same for classification or regression
whole_brain_folder = osp.join(regr_folder, 'whole_brain')
roi_folder = osp.join(clf_folder, 'ROI')

gm_file = osp.join(whole_brain_folder, 's8gm_all.nii')
wm_file = osp.join(whole_brain_folder, 's8wm_all.nii')

mask_scl_mom = load_nifti(osp.join(project_folder, 'data', 'analysis_data', 'masks', 
                                   'mask_overall_new.nii.gz'), dtype=bool)

mask_gm = load_nifti(osp.join(whole_brain_folder, 'mask_s8gm.nii'), dtype=bool)
mask_wm = load_nifti(osp.join(whole_brain_folder, 'mask_s8wm.nii'), dtype=bool)

mask_na = load_nifti(osp.join(roi_folder, 'NucleusAccumbens.nii'), dtype=bool)
mask_atr = load_nifti(osp.join(roi_folder, 'AnteriorThalamicRadiation.nii'), dtype=bool)

gm_3d = load_nifti(gm_file)
wm_3d = load_nifti(wm_file)

X_gm_whole = gm_3d[mask_gm, :].T.copy()
X_wm_whole = wm_3d[mask_wm, :].T.copy()

X_gm_na = gm_3d[mask_na, :].T.copy()
X_wm_atr = wm_3d[mask_atr, :].T.copy()

del gm_3d, wm_3d

df_regr_whole_brain = pd.read_csv(osp.join(whole_brain_folder, 'design', 'design.csv'))
df_clf_roi = pd.read_csv(osp.join(roi_folder, 'design', 'design.csv'))
assert (df_regr_whole_brain.Patient == df_clf_roi.Patient).all()

X_gm_scl, X_wm_scl = load_scalar_momenta(scalar_momenta_folder, 
                                         df_regr_whole_brain.Patient.values, 
                                         mask_scl_mom)

covariate_variables = ['age_baseline', 'gender_1F_0M', 'cat12_TIV_vol_cm3', 
                       'scanner_A', 'scanner_B', 'scanner_C', 'scanner_D', 'scanner_E']
# It was verified manually that the same (!) covariate values were present both in 
# df_clf_roi and df_regr_whole_brain, except for scan type_1 which had to be dropped for 
# the responders vs non-responders design since it would have lead to a singular matrix 
# (sum scan_types == responders + non-responders)
# will be used below in the covariate regression
idx_covariate_demean = [0, 1, 2]

C = df_regr_whole_brain[covariate_variables].values

# Y-BOCS is scaled between 0 and 1 (maximum value is 40)
regr_scaling_factor = 40
y_regr = df_regr_whole_brain.YBOCS_1y.values / regr_scaling_factor
y_clf = df_clf_roi.responder.values 
ybocs_pre = df_regr_whole_brain.YBOCS_baseline.values / regr_scaling_factor

Loading Scalar Momenta...


In [4]:
def get_n_features(n_ftrs, perc_chosen):
    return int(round(perc_chosen * n_ftrs))


def clf_feature_selection(X, y, perc_chosen=0.1):
    # verified against scikit-learns implementation
    n = y.size
    # 2 because of 2 classes in y for degrees of freedom (df) calculation
    df = n - 2
    to_pick = get_n_features(X.shape[1], perc_chosen)
    id_grp1 = (y == 1).astype(np.float)
    id_grp0 = (y == 0).astype(np.float)
    n1 = id_grp1.sum(dtype=np.int)
    n0 = n - n1
    # instead of dividing by n1/n0 and the multiplying by it again
    mu_grp1 = id_grp1.dot(X)
    mu_grp0 = id_grp0.dot(X)
    mu_all = (mu_grp1 + mu_grp0) 
    mu_all /= n
    mu_grp1 /= n1
    mu_grp0 /= n0
    
    # also no normalization as before because of the same reason as before
    sqr_var_grp1 = (X - mu_grp1) ** 2
    var_grp1 = id_grp1.dot(sqr_var_grp1)
    
    sqr_var_grp0 = (X - mu_grp0) ** 2
    var_grp0 = id_grp0.dot(sqr_var_grp0)
    var_all = var_grp0 + var_grp1
    
    fisher_score = (n1 * (mu_grp1 - mu_all) ** 2 + n0 * (mu_grp0 - mu_all) ** 2) 
    fisher_score /= var_all
    fisher_score *= df
    return np.argsort(fisher_score)[-to_pick:]


def regr_feature_selection(X, y, perc_chosen=0.1):
    n = y.size
    to_pick = get_n_features(X.shape[1], perc_chosen)
    
    # correlation calculation verified (multiple times...)
    # the first formula of Pearson correlation from wikipedia
    y = y - y.mean()
    id_all = np.ones(X.shape[0]) / n
    X_means = id_all.dot(X)
    X_norms = np.sqrt(row_norms(X.T, squared=True) - n * X_means ** 2)
    
    r = y.dot(X)
    r /= X_norms
    r /= np.linalg.norm(y)
    return np.argsort(np.abs(r))[-to_pick:]


def feature_selection(analysis_type, X, y, perc_chosen=0.1):
    if analysis_type == 'regression':
        return regr_feature_selection(X, y, perc_chosen=perc_chosen)
    else:
        return clf_feature_selection(X, y, perc_chosen=perc_chosen)

In [5]:
def get_cv(analysis_type='classification', cv_scale='outer', random_state=42):
    
    if analysis_type == 'classification' and cv_scale == 'outer':
        # 10x5-fold CV - stratified
        return RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=random_state)
    
    elif analysis_type == 'classification' and cv_scale == 'inner':
        # 5-fold CV - stratified
        return StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    
    elif analysis_type == 'regression' and cv_scale == 'outer':
        # 10x5-fold CV
        return RepeatedKFold(n_splits=5, n_repeats=10, random_state=random_state)
    
    else:
        # 5-fold CV
        return KFold(n_splits=5, shuffle=True, random_state=random_state)
    
    
def get_ml_model(analysis_type):
    if analysis_type == 'classification':
        return SVC(kernel='linear', C=1.0, probability=True, class_weight='balanced', 
                   tol=1e-4)
    else:
        return SVR(kernel='linear', C=1.0, tol=1e-4)
    
    
def score_inner(model, X, y, analysis_type):
    if analysis_type == 'classification':
        y_score = model.predict_proba(X)[:, 1]
        return roc_auc_score(y_true=y, y_score=y_score)
    else:
        y_pred = model.predict(X)
        # negated here so that we can take the maximum to get the best performing 
        # hyperparameters in both classification and regression
        return -mean_absolute_error(y_true=y, y_pred=y_pred)
    
    
def score_outer(model, X, y, analysis_type, regr_scaling_factor=1):
    y_pred = model.predict(X)
    if analysis_type == 'classification':
        y_score = model.predict_proba(X)[:, 1]
        auc = roc_auc_score(y_true=y, y_score=y_score)
        acc = balanced_accuracy_score(y_true=y, y_pred=y_pred)
        sens = recall_score(y_true=y, y_pred=y_pred, pos_label=1)
        spec = recall_score(y_true=y, y_pred=y_pred, pos_label=0)
        brier = brier_score_loss(y_true=y, y_prob=y_score)
        score_dict = {'ACC': acc, 'AUC': auc, 'SENS': sens, 'SPEC': spec, 'Brier': brier}
    else:
        # done like this on purpose to create a copy and not change anything weird in the main code
        # allows to express all errors in the original Y-BOCS scale (0 to 40)
        y = y * regr_scaling_factor
        y_pred = y_pred * regr_scaling_factor
        mae = mean_absolute_error(y_true=y, y_pred=y_pred)
        mse = mean_squared_error(y_true=y, y_pred=y_pred)
        rmse = np.sqrt(mse)
        medae = median_absolute_error(y_true=y, y_pred=y_pred)
        r2 = r2_score(y_true=y, y_pred=y_pred)
        r = np.corrcoef(y, y_pred)[0, 1]
        score_dict = {'MAE': mae, 'MedAE': medae, 'MSE': mse, 'RMSE': rmse, 'R2': r2, 'r': r}
    return score_dict


def scale_data(X_train, X_test):
    scaler = StandardScaler(with_mean=True, with_std=True)
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train, X_test, scaler


def denoise_data(X_train, X_test, C_train, C_test):
    coefs, _, _, _ = np.linalg.lstsq(C_train, X_train, rcond=None)
    X_train -= C_train.dot(coefs)
    X_test -= C_test.dot(coefs)
    return X_train, X_test, coefs


def one_cv_run(X_train, y_train, C_train, X_test, y_test, C_test, sel_ftrs, idx_covariate_demean, 
               regress_covariates, analysis_type, additional_feature=None):
    if regress_covariates:
        (C_train[:, idx_covariate_demean], C_test[:, idx_covariate_demean], _) = scale_data(
            C_train[:, idx_covariate_demean], C_test[:, idx_covariate_demean])
        X_train, X_test, _ = denoise_data(X_train, X_test, C_train, C_test)

    # only do feature selection if we are actually selecting features
    if sel_ftrs < 1:
        id_selected = feature_selection(analysis_type=analysis_type, 
                                        X=X_train, 
                                        y=y_train, 
                                        perc_chosen=sel_ftrs)
        X_train, X_test = X_train[:, id_selected], X_test[:, id_selected]
    
    X_train = np.column_stack((X_train, additional_feature['train']))
    X_test = np.column_stack((X_test, additional_feature['test']))
        
    X_train, X_test, _ = scale_data(X_train, X_test)

    ml_model = get_ml_model(analysis_type)
    ml_model.fit(X_train, y_train)
    
    # y_test hasn't be touched but will be passed through 
    return X_test, y_test, ml_model,


def get_data_types_names():
    return ['whole-brain:GMV', 'whole-brain:WMV', 'whole-brain:GMV+WMV', 'ROI:NAc', 'ROI:ATR', 'ROI:NAc+ATR',
            'SclMom:GM+WM']


def get_perf_metrics(analysis_type):
    if analysis_type == 'classification':
        perf_metrics = ['ACC', 'AUC', 'SENS', 'SPEC', 'Brier'] 
    else:
        perf_metrics = ['MAE', 'MedAE', 'MSE', 'RMSE', 'R2', 'r']
    return perf_metrics


def get_df_scores(analysis_type):
    index = get_data_types_names()
    stats = ['mean', 'SD']
    
    perf_metrics = get_perf_metrics(analysis_type)
    multi_col_index = pd.MultiIndex.from_product([perf_metrics, stats])
    df = pd.DataFrame(index=index, columns=multi_col_index)
    return df


def get_df_cv(analysis_type):
    perf_metrics = get_perf_metrics(analysis_type)
    if analysis_type == 'classification':
        df = pd.DataFrame(columns=perf_metrics)
    else:
        df = pd.DataFrame(columns=perf_metrics)
    return df

In [7]:
data_dict = {'whole-brain': {'GMV': X_gm_whole, 
                             'WMV': X_wm_whole, 
                             'GMV+WMV': np.column_stack((X_gm_whole, X_wm_whole))},
             'ROI': {'NAc': X_gm_na, 
                     'ATR': X_wm_atr, 
                     'NAc+ATR': np.column_stack((X_gm_na, X_wm_atr))},
             'SclMom': {'GM+WM': np.column_stack((X_gm_scl, X_wm_scl))}}

dtype_names = get_data_types_names()

analysis_dict = {'classification': y_clf,
                 'regression': y_regr}

perc_chosen = {'whole-brain': [0.001, 0.005, 0.01, 0.05, 0.1, 1],
               'ROI': [0.01, 0.05, 0.1, 0.15, 0.20, 1],
               'SclMom': [0.001, 0.005, 0.01, 0.05, 0.1, 1]}

df_perc_chosen = pd.DataFrame(index=dtype_names, 
                              columns=[0.001, 0.005, 0.01, 0.05, 0.1, 0.15, 0.20, 1])

results_dict = {}

for analysis_type, y in analysis_dict.items():
    print(analysis_type)
    
    random_state_outer = int(time())
    random_state_inner = random_state_outer + 100
    df_score = get_df_scores(analysis_type)

    for data_scale, X_dict in data_dict.items():
        print(data_scale)
        
        for data_type, X in X_dict.items():
            print(data_type)
            
            cv_outer = get_cv(analysis_type=analysis_type, cv_scale='outer', 
                              random_state=random_state_outer)
            
            chosen_probabilities = perc_chosen[data_scale]
            
            df_cv = get_df_cv(analysis_type)
            picked_probs = pd.Series(data=0, index=chosen_probabilities)

            for i_cv_outer, (id_train, id_test) in enumerate(tqdm(cv_outer.split(X, y))):
                X_train, X_test = X[id_train], X[id_test]
                C_train, C_test = C[id_train], C[id_test]
                y_train, y_test = y[id_train], y[id_test]
                ybocs_pre_train, ybocs_pre_test = ybocs_pre[id_train], ybocs_pre[id_test]
                ybocs_pre_outer = {'train': ybocs_pre_train,
                                   'test': ybocs_pre_test}
                
                scores_grid_search = np.zeros((5, len(chosen_probabilities)))
            
                for i_ftrs, sel_ftrs in enumerate(chosen_probabilities):
                    cv_inner = get_cv(analysis_type=analysis_type, cv_scale='inner', 
                                      random_state=random_state_inner)
                    
                    for i_cv_inner, (id_train_inner, id_test_inner) in enumerate(cv_inner.split(X_train, 
                                                                                                y_train)):
                        X_train_inner, X_test_inner = (X_train[id_train_inner], 
                                                       X_train[id_test_inner])
                        C_train_inner, C_test_inner = (C_train[id_train_inner], 
                                                       C_train[id_test_inner])
                        y_train_inner, y_test_inner = (y_train[id_train_inner], 
                                                       y_train[id_test_inner])
                        ybocs_pre_train_inner, ybocs_pre_test_inner = (ybocs_pre_train[id_train_inner],
                                                                       ybocs_pre_train[id_test_inner])
                        
                        ybocs_pre_inner = {'train': ybocs_pre_train_inner,
                                           'test': ybocs_pre_test_inner}
                        
                        X_test_inner, y_test_inner, ml_model = one_cv_run(X_train_inner, 
                                                                          y_train_inner, 
                                                                          C_train_inner, 
                                                                          X_test_inner, 
                                                                          y_test_inner, 
                                                                          C_test_inner, 
                                                                          sel_ftrs, 
                                                                          idx_covariate_demean, 
                                                                          regress_covariates, 
                                                                          analysis_type,
                                                                          additional_feature=ybocs_pre_inner)
    
                        
                        scores_grid_search[i_cv_inner, i_ftrs] = score_inner(ml_model, 
                                                                             X_test_inner, 
                                                                             y_test_inner, 
                                                                             analysis_type)
                
                mean_scores_gsearch = scores_grid_search.mean(axis=0)
                id_best = mean_scores_gsearch.argmax()
                best_prob = chosen_probabilities[id_best]
                picked_probs[best_prob] += 1
                
                X_test, y_test, ml_model = one_cv_run(X_train, 
                                                      y_train, 
                                                      C_train, 
                                                      X_test, 
                                                      y_test, 
                                                      C_test, 
                                                      best_prob, 
                                                      idx_covariate_demean, 
                                                      regress_covariates, 
                                                      analysis_type, 
                                                      additional_feature=ybocs_pre_outer)
                
                df_cv = df_cv.append(score_outer(ml_model, X_test, y_test, analysis_type, 
                                                 regr_scaling_factor=regr_scaling_factor), ignore_index=True)
            
            df_perc_chosen.loc["{}:{}".format(data_scale, data_type)] = picked_probs
            
            mean_cv = df_cv.mean()
            sd_cv = df_cv.std()
            index_cv = mean_cv.index
            assert (index_cv == sd_cv.index).all()
            
            df_cv.to_csv('results_cv_{}_{}_{}_{}.csv'.format(analysis_type, 
                                                             data_type, 
                                                             data_scale, 
                                                             suffix_cov_reg), index=False)
            
            if 'r' in index_cv:
                r = df_cv['r'].values
                rtoz = np.arctanh(r)
                mean_r = np.tanh(rtoz.mean())
                sd_r = np.tanh(rtoz.std())
                mean_cv.loc['r'] = mean_r
                sd_cv.loc['r'] = sd_r
            
            df_score.loc['{}:{}'.format(data_scale, data_type), 
                         (index_cv, 'mean')] = mean_cv.values
            df_score.loc['{}:{}'.format(data_scale, data_type), 
                         (index_cv, 'SD')] = sd_cv.values
    
    df_score[df_perc_chosen.columns] = df_perc_chosen
    df_score['random_seed'] = random_state_outer
    df_score.to_csv('results_{}_{}.csv'.format(analysis_type, suffix_cov_reg))
    results_dict[analysis_type] = df_score
    print()
    print(analysis_type)
    print(df_score)    
    print()

classification
whole-brain
GMV



WMV



GMV+WMV



ROI
NAc



ATR



NAc+ATR



SclMom
GM+WM




classification
                          ACC                 AUC                SENS  \
                         mean        SD      mean        SD      mean   
whole-brain:GMV      0.482143  0.102543   0.46781   0.12418  0.567619   
whole-brain:WMV      0.448952  0.145226   0.42227  0.172875  0.575238   
whole-brain:GMV+WMV     0.481  0.155608  0.433222  0.170585  0.606667   
ROI:NAc              0.572952  0.132347  0.542746  0.213211  0.621905   
ROI:ATR              0.428619  0.148593  0.509286  0.201441  0.471905   
ROI:NAc+ATR          0.485714   0.14566  0.480556  0.161548  0.541429   
SclMom:GM                 NaN       NaN       NaN       NaN       NaN   
SclMom:WM                 NaN       NaN       NaN       NaN       NaN   
SclMom:GM+WM         0.515238  0.102409  0.546016  0.136442   0.56381   

                                   SPEC               Brier            0.001  \
                           SD      mean        SD      mean         SD         
whole-brain:GMV    

In [ ]:
print('MAE: ', np.mean(np.abs(y_regr - y_regr.mean())))
print('MedAE: ', np.median(np.abs(y_regr - y_regr.mean())))
print('MSE: ', np.mean((y_regr - y_regr.mean()) ** 2))
print('RMSE: ', np.sqrt(np.mean((y_regr - y_regr.mean()) ** 2)))

In [ ]:
print('MAE: ', np.mean(np.abs(y_regr * 100 - y_regr.mean() * 100)))
print('MedAE: ', np.median(np.abs(y_regr * 100 - y_regr.mean() * 100)))
print('MSE: ', np.mean((y_regr * 100 - y_regr.mean() * 100) ** 2))
print('RMSE: ', np.sqrt(np.mean((y_regr * 100 - y_regr.mean() * 100) ** 2)))

#### Running permutation tests only for the classification case with ROIs

In [ ]:
data_dict = {'NAc': X_gm_na, 
             'ATR': X_wm_atr, 
             'NAc+ATR': np.column_stack((X_gm_na, X_wm_atr))}

analysis_type = 'classification'
chosen_probabilities = [0.01, 0.05, 0.1, 0.15, 0.20, 1]
n_perm = 1000
perf_metrics = get_perf_metrics(analysis_type)

print('Running permutation tests:')

for roi_name, X in data_dict.items():
    print(roi_name)
    
    df_perm = pd.DataFrame(columns=perf_metrics)
    y_perm = y_clf.copy()
    seed = int(time())
    rng = np.random.RandomState(seed)
    
    for i_perm in tqdm(range(n_perm)):
        y_perm = rng.permutation(y_perm)
        
        random_state_outer = seed + 100
        random_state_inner = random_state_outer + 100
        df_score = get_df_scores(analysis_type) 
        cv_outer = get_cv(analysis_type=analysis_type, cv_scale='outer', 
                          random_state=random_state_outer)
                       
        df_cv = get_df_cv(analysis_type)
        
        for i_cv_outer, (id_train, id_test) in enumerate(cv_outer.split(X, y_perm)):
            X_train, X_test = X[id_train], X[id_test]
            C_train, C_test = C[id_train], C[id_test]
            y_train, y_test = y_perm[id_train], y_perm[id_test]

            scores_grid_search = np.zeros((5, len(chosen_probabilities)))
            
            for i_ftrs, sel_ftrs in enumerate(chosen_probabilities):
                cv_inner = get_cv(analysis_type=analysis_type, cv_scale='inner', 
                                  random_state=random_state_inner)
                    
                for i_cv_inner, (id_train_inner, id_test_inner) in enumerate(cv_inner.split(X_train, 
                                                                                                y_train)):
                    X_train_inner, X_test_inner = (X_train[id_train_inner], 
                                                   X_train[id_test_inner])
                    C_train_inner, C_test_inner = (C_train[id_train_inner], 
                                                   C_train[id_test_inner])
                    y_train_inner, y_test_inner = (y_train[id_train_inner], 
                                                   y_train[id_test_inner])
                        
                    X_test_inner, y_test_inner, ml_model = one_cv_run(X_train_inner, 
                                                                      y_train_inner, 
                                                                      C_train_inner, 
                                                                      X_test_inner, 
                                                                      y_test_inner, 
                                                                      C_test_inner, 
                                                                      sel_ftrs, 
                                                                      idx_covariate_demean, 
                                                                      regress_covariates, 
                                                                      analysis_type)
    
                        
                    scores_grid_search[i_cv_inner, i_ftrs] = score_inner(ml_model, 
                                                                         X_test_inner, 
                                                                         y_test_inner, 
                                                                         analysis_type)
            
            mean_scores_gsearch = scores_grid_search.mean(axis=0)
            id_best = mean_scores_gsearch.argmax()
            best_prob = chosen_probabilities[id_best]
                
            X_test, y_test, ml_model = one_cv_run(X_train, 
                                                  y_train, 
                                                  C_train, 
                                                  X_test, 
                                                  y_test, 
                                                  C_test, 
                                                  best_prob, 
                                                  idx_covariate_demean, 
                                                  regress_covariates, 
                                                  analysis_type)
                
            df_cv = df_cv.append(score_outer(ml_model, X_test, y_test, analysis_type), 
                                 ignore_index=True)
                        
        mean_cv = df_cv.mean()
        df_perm = df_perm.append(mean_cv, ignore_index=True)

    df_perm.to_csv('results_perm_{}_{}.csv'.format(analysis_type, roi_name), index=False)

In [ ]:
def calculate_permutation_pvalue(df_real, df_perm, metric_to_use='AUC'):
    real_value = df_real[metric_to_use, 'mean']
    perm_values = df_perm[metric_to_use].to_numpy()
    print('Real value: {:0.4f}'.format(real_value))
    print('Permutation values: mean: {:0.4f}; min: {:0.4f}; max: {:0.4f}'.format(perm_values.mean(),
                                                                                 perm_values.min(),
                                                                                 perm_values.max()))
    n_perm = df_perm.shape[0] + 1  # plus the neutral permutation
    return (np.sum(perm_values >= real_value) + 1) / n_perm

df_result_clf = pd.read_csv('results_classification_with_covariate_regr.csv', index_col=0, 
                            header=[0, 1])
df_result_clf_roi = df_result_clf.loc[['ROI:NAc', 'ROI:ATR', 'ROI:NAc+ATR']]
roi_names = ['NAc', 'ATR', 'NAc+ATR']
metric_to_use = 'AUC'
n_tests = 3

print('Metrics to calculate p-value for: {}'.format(metric_to_use))
for roi_name in roi_names:
    print(roi_name)
    df_perm_roi = pd.read_csv('results_perm_classification_{}.csv'.format(roi_name))
    p_roi = calculate_permutation_pvalue(df_result_clf_roi.loc['ROI:{}'.format(roi_name)],
                                         df_perm_roi, metric_to_use)
    print('P-value: {:0.6f}'.format(p_roi))
    print('P-value - Bonferroni-corrected for {} tests: {:0.6f}'.format(n_tests, p_roi * n_tests))
    print()